# 1 - Imports

In [1]:
import sys
sys.path.append('/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/scripts/')

import argparse
import librosa
import librosa.display as ld
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from model_evaluator_2 import ModelEvaluator
from settings import MODEL_EVALUATOR_DEFAULT_SETTINGS

# 2 - Feature extractor class

In [2]:
default_params_dict = MODEL_EVALUATOR_DEFAULT_SETTINGS

default_params_dict['model_checkpoint_path'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/models/v1_tiny/v1_tiny_VGGNL_DoubleMHA_700.chkpt'
default_params_dict['test_clients'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/clients.ndx'
default_params_dict['test_impostors'] = '/home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/impostors.ndx'
default_params_dict['dump_folder'] = './models_results/v1___'
default_params_dict['data_dir'] = ['/home/usuaris/veussd/DATABASES/VoxCeleb/VoxCeleb1/test/']

default_params = argparse.Namespace(**default_params_dict)

model_evaluator_2 = ModelEvaluator(default_params)

22-10-06 13:09:16 - model_evaluator_2 - INFO - Setting device...
22-10-06 13:09:17 - model_evaluator_2 - INFO - Running on cuda device.
22-10-06 13:09:17 - model_evaluator_2 - INFO - Device setted.
22-10-06 13:09:17 - model_evaluator_2 - INFO - Setting random seed...
22-10-06 13:09:17 - model_evaluator_2 - INFO - Random seed setted.
22-10-06 13:09:17 - model_evaluator_2 - INFO - Loading checkpoint from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/models/v1_tiny/v1_tiny_VGGNL_DoubleMHA_700.chkpt
22-10-06 13:09:18 - model_evaluator_2 - INFO - Model checkpoint was saved at epoch 0
22-10-06 13:09:18 - model_evaluator_2 - INFO - Checkpoint loaded.
22-10-06 13:09:19 - model_evaluator_2 - INFO - Loading data from /home/usuaris/veu/federico.costa/git_repositories/DoubleAttentionSpeakerVerification/labels/test/tiny_check/clients.ndx
22-10-06 13:09:19 - model_evaluator_2 - INFO - Loading data from /home/usuaris/veu/federico.costa/git_repositories/DoubleAtt

# 3 - Tests

In [3]:
with torch.no_grad():

    # Switch torch to evaluation mode
    model_evaluator_2.net.eval()

    for model_evaluator_2.batch_number, (input_1, len_1, label) in enumerate(model_evaluator_2.training_generator):

        print(f"Batch {model_evaluator_2.batch_number} of {model_evaluator_2.total_batches}")

        input_1 = input_1.float().to(model_evaluator_2.device)
        label = label.int().to(model_evaluator_2.device)

        break

Batch 0 of 1


In [4]:
input_1.size()

torch.Size([36, 350, 80])

In [5]:
len_1

tensor([350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350,
        350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350,
        350, 350, 350, 350, 350, 350, 350, 350])

In [6]:
label.size()

torch.Size([36])

In [7]:
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first=True)
b



tensor([[1, 2, 3],
        [3, 4, 0]])

In [8]:
torch.nn.utils.rnn.pack_padded_sequence(b, batch_first=True, lengths=[3,2])

PackedSequence(data=tensor([1, 3, 2, 4, 3]), batch_sizes=tensor([2, 2, 1]), sorted_indices=None, unsorted_indices=None)

In [25]:
data = [
    (input_1[0], label[0], len_1[0]), 
    (input_1[1], label[1], len_1[1])
]

_, labels, lengths = zip(*data)
max_len = max(lengths)
n_ftrs = data[0][0].size(1)
features = torch.zeros((len(data), max_len, n_ftrs))
labels = torch.tensor(labels)
lengths = torch.tensor(lengths)

for i in range(len(data)):
    j, k = data[i][0].size(0), data[i][0].size(1)
    features[i] = torch.cat([data[i][0], torch.zeros((max_len - j, k))])

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument tensors in method wrapper_cat)

In [23]:
features

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])